In [ ]:
import os
import numpy as np
import torch as pt
import pandas as pd
from tqdm import tqdm
from glob import glob
from time import time

import src as sp
import runtime as rt

In [ ]:
# parameters
device = pt.device("cuda")
n_sample = 1

# locate filepaths
pdb_filepaths = glob("tmp/*.pdb")

# model parameters
save_path = "model/save/s_v6_4_2022-09-16_11-51"  # virtual Cb & partial

# create runtime model 
model = rt.SequenceModel(save_path, "model.pt", device=device)

In [ ]:
profiling = []
for pdb_filepath in tqdm(pdb_filepaths):
    t0 = time()
    # load structure
    structure = rt.load_structure(pdb_filepath)
    
    # max size
    if structure['xyz'].shape[0] > 1024*8:
        continue
    
    # apply model
    structure, p, y, _ = model(structure)

    # amino acid only
    p, y = rt.aa_only(p, y)
    
    # get sequence
    seq = rt.max_pred_to_seq(p)
    
    # profiling
    t1 = time()
    sid = os.path.basename(pdb_filepath).split('.')[0]
    profiling.append({'sid': sid, 'dt': t1-t0})
    
# save profiling
dfp = pd.DataFrame(profiling)
dfp.to_csv("results/profiling_carbonara_ext.csv", index=False)